In [1]:
import os

In [2]:
%pwd

'd:\\Projects\\Wine-Quality---ML-Project-with-MLFlow\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\Projects\\Wine-Quality---ML-Project-with-MLFlow'

In [5]:
import pandas as pd

In [36]:
data = pd.read_csv("artifacts\data_ingestion\winequalityN.csv")
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6497 entries, 0 to 6496
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   type                  6497 non-null   object 
 1   fixed acidity         6487 non-null   float64
 2   volatile acidity      6489 non-null   float64
 3   citric acid           6494 non-null   float64
 4   residual sugar        6495 non-null   float64
 5   chlorides             6495 non-null   float64
 6   free sulfur dioxide   6497 non-null   float64
 7   total sulfur dioxide  6497 non-null   float64
 8   density               6497 non-null   float64
 9   pH                    6488 non-null   float64
 10  sulphates             6493 non-null   float64
 11  alcohol               6497 non-null   float64
 12  quality               6497 non-null   int64  
dtypes: float64(11), int64(1), object(1)
memory usage: 660.0+ KB


In [47]:
data.shape

(6497, 13)

In [48]:
from pathlib import Path
from dataclasses import dataclass

@dataclass(frozen=True)
class DataValidationConfig:
    root_dir: Path
    STATUS_FILE: str
    unzip_data_dir: Path
    all_schema: dict

In [49]:
from src.MLProject.constants import *
from src.MLProject.utils.common import read_yaml, create_directories


In [50]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])
        
    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config.data_validation
        schema = self.schema.COLUMNS

        create_directories([config.root_dir])

        data_validation_config=DataValidationConfig(
            root_dir = config.root_dir,
            STATUS_FILE=config.STATUS_FILE,
            unzip_data_dir = config.unzip_data_dir,
            all_schema=schema
        )

        return data_validation_config

In [51]:
import os
from src.MLProject import logger


In [52]:
class DataValidation:
    def __init__(self, config: DataValidationConfig):
        self.config = config 

    def validate_all_columns(self) -> bool:
        try:
            validate_status = None

            data = pd.read_csv(self.config.unzip_data_dir)
            all_cols = list(data.columns)

            all_schema = self.config.all_schema.keys()

            for col in all_cols:
                if col not in all_schema:
                    validation_status = False
                    with open(self.config.STATUS_FILE,'w') as f:
                        f.write(f"Validation Status: {validation_status}")

                else:
                    validation_status = True
                    with open(self.config.STATUS_FILE,'w') as f:
                        f.write(f"Validation status: {validation_status}")

            return validation_status
        
        except Exception as e:
            raise e


In [53]:
try:
    config = ConfigurationManager()
    data_validation_config =config.get_data_validation_config()
    data_validation = DataValidation(config=data_validation_config)
    data_validation.validate_all_columns()
except Exception as e:
    raise e

[2024-01-09 13:24:34,380: INFO: common: yaml file: config\config.yaml load successfully]
[2024-01-09 13:24:34,381: INFO: common: yaml file: params.yaml load successfully]
[2024-01-09 13:24:34,385: INFO: common: yaml file: schema.yaml load successfully]
[2024-01-09 13:24:34,386: INFO: common: created directory at: artifacts]
[2024-01-09 13:24:34,389: INFO: common: created directory at: artifacts/data_validation]
